## Constructing a Network of Wikipedia Pages

Wikipedia page body has external links and links to other Wikipedia pages. To build a network out of the seed page and other relevant pages, let’s treat
the pages as the network nodes and the links between the pages as the network edges. We will use snowball sampling to discover all the nodes and edges of interest. Our seed page is **"complex network"** .

In [2]:
#the operator itemgetter for sorting a list of tuples
from operator import itemgetter
import networkx as nx
import wikipedia

SEED = "Complex network".title()

# ignore any links to them
STOPS = ("International Standard Serial Number",
         "International Standard Book Number",
         "National Diet Library",
         "International Standard Name Identifier",
         "International Standard Book Number (Identifier)",
         "Pubmed Identifier", "Pubmed Central",
         "Digital Object Identifier", "Arxiv",
         "Proc Natl Acad Sci Usa", "Bibcode",
         "Library Of Congress Control Number", "Jstor")

In [3]:
todo_lst = [(0, SEED)] # The SEED is in the layer 0
todo_set = set(SEED)   # The SEED itself
done_set = set()       # Nothing is done yet

In [4]:
# edges that represent HTML links are naturally directed: a link from page A to page B
F = nx.DiGraph()

# extracting the first “to-do” item
layer, page = todo_lst[0]

* we will process only the seed node itself and its immediate neighbors (layers 0 and 1).

In [5]:
while layer < 2:
    del todo_lst[0] #(1)
    done_set.add(page)
#     print(layer, page) # Show progress

    try: #(2)
        wiki = wikipedia.page(page)
    except:
        layer, page = todo_lst[0]
        print("Could not load", page)
        continue

    for link in wiki.links: #(3)
        link = link.title()
        
        # exclud pages whose names begin with "List of", because they are simply lists of other subjects.
        if link not in STOPS and not link.startswith("List Of"):
            if link not in todo_set and link not in done_set:
                todo_lst.append((layer + 1, link))
                todo_set.add(link)
            F.add_edge(page, link)

    layer, page = todo_lst[0] #(4)
print("{} nodes, {} edges".format(len(F), nx.number_of_edges(F)))

Could not load Power-Law
Could not load Sir Model
12391 nodes, 23729 edges


In [8]:
# removing self-loops (pages referring to themselves)
F.remove_edges_from(list(nx.selfloop_edges(F)))

# build duplicates by looking at each node in F and checking if a node with the same name, but
# with an s at the end, is also in F.
duplicates = [(node, node + "s") for node in F if node + "s" in F]

# merges node v into node u in the graph F. The function reassigns all edges previously incident to v, to u. If you don’t
# pass the option self_loops=False, the function converts an edge from v to u (if
# any) to a self-loop.
for dup in duplicates:
    F = nx.contracted_nodes(F, *dup, self_loops=False)

In [9]:
# checking if a node with the same name, but hyphen in the middle
duplicates = [(x, y) for x, y 
              in [(node, node.replace("-", " ")) for node in F]
              if x != y and y in F]

for dup in duplicates:
    F = nx.contracted_nodes(F, *dup, self_loops=False)

* As a side effect, `nx.contracted_nodes()` creates a new node attribute called contraction. The value of the attribute is a dictionary, but GraphML does not support dictionary attributes. So, we set the contraction property to 0 for all nodes to avoid further troubles with exporting the graph. You could also delete the attribute for each node n with del n["contraction"] in a loop.

In [10]:
nx.set_node_attributes(F, 0, "contraction")

***node indegree :*** the number of edges directed into the node.
the number of edges directed out of the node is called **outdegree**.

The indegree of a node equals the number of HTML links pointing to the respective page. If a page has a lot of links to it, the
topic of the page must be significant.

We will remove all nodes with only one incident edge to make the network more compact and less complex. `F.degree()` method returns a dictionary with nodes as keys and
degree as values.

In [11]:
core = [node for node, deg in dict(F.degree()).items() if deg >= 2]

Function `nx.subgraph(F, core)` collects all core nodes from F and all edges connecting
them and builds a new graph G—a subgraph of F.

In [12]:
G = nx.subgraph(F, core)

print("{} nodes, {} edges".format(len(G), nx.number_of_edges(G)))

2872 nodes, 13179 edges


Write it to a GraphML file so that you don’t have to rebuild it if you need it later again.

In [16]:
nx.write_graphml(G, "cna_1.graphml")

In [14]:
top_indegree = sorted(dict(G.in_degree()).items(),
                      reverse=True, key=itemgetter(1))[:10]
print("\n".join(map(lambda t: "{} {}".format(*reversed(t)), top_indegree)))

68 Graph (Discrete Mathematics)
65 Vertex (Graph Theory)
58 Directed Graph
55 Social Network
52 Network Theory
52 Degree (Graph Theory)
50 Graph Drawing
49 Complete Graph
49 Edge (Graph Theory)
49 Graph (Abstract Data Type)


In [15]:
top_indegree = sorted(G.in_degree(),
                      reverse=True, key=itemgetter(1))[:10]
print("\n".join(map(lambda t: "{} {}".format(*reversed(t)), top_indegree)))

68 Graph (Discrete Mathematics)
65 Vertex (Graph Theory)
58 Directed Graph
55 Social Network
52 Network Theory
52 Degree (Graph Theory)
50 Graph Drawing
49 Complete Graph
49 Edge (Graph Theory)
49 Graph (Abstract Data Type)


Source : Dmitry Zinoviev - Complex Network Analysis in Python